Loading Groq Model


In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [2]:
from langchain_groq import ChatGroq

llamaChatModel = ChatGroq(
    model="llama-3.3-70b-versatile"
)

#Prompts and Prompt template

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an {profession} expert on {topic}",
        ),
        ("human", "Hello Mr {profession},can you please answer a question"),
        ("ai","Sure!"),
        ("human","{user_input}")
    ]
)

messages=prompt.format_messages(
    profession="Historian",
    topic="The kennedy family",
    user_input="How many grandchildren does Joseph P. Kennedy have ?"
)

response=llamaChatModel.invoke(messages)
print(response.content)

Joseph P. Kennedy, the patriarch of the Kennedy family, had 28 grandchildren. He had nine children, and those children went on to have a total of 28 grandchildren.


Few shot examples with chaining 

In [8]:
# Creating a language transalation instructor
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "¡hola!"},
    {"input": "bye!", "output": "¡adiós!"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

chain = final_prompt | llamaChatModel

res = chain.invoke({"input": "How are you?"})
print(res.content)

Estoy bien, ¿y tú? (I'm fine, and you?)


Output parsers

In [10]:
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
json_prompt = ChatPromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_prompt | llamaChatModel | json_parser

In [11]:
res  = json_chain.invoke({"question": "What is the biggest country?"})

In [12]:
res

{'answer': 'Russia'}